In [1]:
# Binary integer program (BIP) that assigns carry-in sites to prep sites
# This model does not consider delivery windows or vehicle capacity, those constraints will be considered later
# Input: supply of prep sites, demand of carry-in sites, cost of going from prep to carry-in site (time,distance,whatever)
# Output: pairings with carry-in sites listed first

from ortools.linear_solver import pywraplp
import pandas as pd 

In [2]:
# Notes
# Don't leave 0 demand or supply entries in the initial Excel files (or edit code to handle 0 demand/supply)
## Could make this into a function that takes file paths as the input; format of files would need to be uniform 

In [3]:
## Model 
# Declare solver   
solver = pywraplp.Solver('SolveAssignmentProblemMIP',pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

In [4]:
## Helper functions

def convert_to_dict(keylist,valuelist):
#makes two lists into a dictionary with the first list as the keys
#list lengths must be equivalent
#Replace with dict comprehension? 
    temp_dict = {}
    if len(keylist) != len(valuelist):
        print("List lengths not equivalent")
        return
    for x in range(len(keylist)):
        temp_dict[keylist[x]] = valuelist[x]
    return temp_dict

def missing_key(master_dict,subset_dict):
#returns keys that exist in master_dict but not in subset_dict
#here master_dict will be segments and subset_dict will be costs
#costs generated from current arcs but segments is list of all potential arcs
#use to find missing keys if getting key error when running objective function cell
    missing_keys = {}
    for key in master_dict.keys():
        if key not in subset_dict.keys():
            missing_keys[key] = 1
    return missing_keys

def list_to_tuple(lst):
# makes a list of lists into a list of tuples 
# [[a,b],[c,d],[e,f],[g,h]] -> [(a,b),(c,d),(e,f),(g,h)]
    for i in range(len(lst)):
        lst[i] = tuple(lst[i])
    return lst

def amount_used():
# a dictionary with prep sites as the key, pointing to how many meals they are assigned to serve
    num_meals = {}
    for j in prep_site_number:
        used = [0]
        for i in carry_in_site_number:
            if (i,j) in assignments.keys(): ## only in assignments if assigned to each other
                used[0] = used[0] + carry_in_dict[i]
        num_meals[j] = used[0]
    return num_meals

def make_prep_list(assignments_dict):
# takes dictionary of assignments which has a tuple key and binary value where the key is the prep/carry-in match
# makes it into a different dict prep_list which has the carry ins assigned to each prep as a list 
# (prep1,carryin1:1), (prep1,carryin2:1) -> prep1: [carryin1,carryin2]
    unique_prep = []
    for key in assignments_dict.keys():
        prep = key[1]
        if prep not in unique_prep:
            unique_prep.append(prep)
    
    prep_list = {}
    for p in unique_prep:
        prep_list[p] = []
        # key p now points to empty list
        for key in assignments_dict.keys():
            if p == key[1]:
                prep_list[p].append(key[0])
                ## take k[0] and add it to list that p points to in prep_list
    
    return prep_list

def check_key(dict,mykey):
# checks if some key mykey is in dictionary dict
    if mykey in dict:
        print("Present")
    else:
        print("No")
        
def remove_zero_value_dict(dict):
# remove all 0 values from your dictionary
    new_dict = {} 
    for key in dict.keys():
        if dict[key] != 0:
            new_dict[key] = dict[key]
    return new_dict

def assignments_different(assignments_dict1, assignments_dict2):
    # returns the keys that point to different values in each dictionary
    # Useful if comparing different inputted data / how it impacts assignments
    differences = {}
    for key in assignments_dict1.keys():
        if assignments_dict1[key] != assignments_dict2[key]:
            differences[key] = 1
            
    return differences

In [5]:
## Read in data
## change computer path as necessary
## Change column names in Excel file or here accordingly, remember to save Excel file before running again

# Supply
supply_df = pd.read_excel("C://Users/anyak/Documents/DPS/DPSData/ImportantDataCompiled.xlsx", "Prep Site Capacity",usecols = "B,C")
# pd.read_excel('filepath','worksheet in question', 'columns in question')
supply_df = supply_df.dropna().astype(int) # drop null values and convert data type to int
prep_site_number = supply_df["Prep Site #"].values.tolist() 
supply = supply_df["Supply"].values.tolist() # makes column of dataframe into list

# Demand (using different numbering system to differentiate lunch and breakfast at same school)
demand_df = pd.read_excel("C://Users/anyak/Documents/DPS/DPSData/DuplicateSchoolCodes.xlsx",usecols = "A,B")
carry_in_site_number = demand_df["School"].dropna().values.tolist() 
demand = demand_df["Demand"].dropna().astype(int)

# Distances
distances_df = pd.read_excel("C://Users/anyak/Documents/DPS/DPSData/preptocarryLATEST2.xlsx",usecols = "C,D,E")
cost_list = distances_df["Total_Kilometers"].values.tolist()

# Destinations (carry ins) first for consistency going forward
arcs = distances_df[["Dest_name","Org_name"]].values.tolist() ## list of lists, need to convert to list of tuples
arcs = list_to_tuple(arcs)

In [6]:
# turn lists into dictionaries
carry_in_dict = convert_to_dict(carry_in_site_number,demand)
prep_site_dict = convert_to_dict(prep_site_number,supply)
costs = convert_to_dict(arcs,cost_list)

In [7]:
## Declare binary decision variables
#i by j array segments where (i,j) is keyed to 1 if carry_in i is assigned to prep j and 0 otherwise
#segments is a dictionary which takes the tuple (i,j) as the key and the bool as the value
segments = {}
for i in carry_in_site_number:
    for j in prep_site_number:
        segments[(i, j)] = solver.BoolVar('seg[%i,%i]' % (i, j))

In [8]:
# Check if there are any pairings where there is no distance information
# if empty, good to go
missing_key(segments,costs)

{}

In [9]:
## Objective: minimize total distance between prep and carry sites 
# is not optimal minimization of drive time / miles driven, but ensures proximity heuristically
solver.Minimize(solver.Sum([costs[(i,j)]*segments[(i,j)]
                                        for i in carry_in_site_number
                                        for j in prep_site_number]))

In [10]:
## Constraints
## Another possible constraint: maybe look for time windows not to overlap 

# carry in i is assigned at least and no more than one prep j (prep j can have multiple carry_ins)
for i in carry_in_site_number:
    solver.Add(solver.Sum([segments[(i,j)] for j in prep_site_number]) == 1)

# sum of demand of carry_ins I assigned to j do not exceed supply of prep j
# carry_in_site_dict[i] gives demand of site i
# prep_site_dict[j] gives supply of prep j
for j in prep_site_number:
    solver.Add(solver.Sum([segments[(i,j)]*carry_in_dict[i] for i in carry_in_site_number]) <= prep_site_dict[j])
    

## Potential additional constraint    
# Each prep site must service at least one carry in site
# More even spread of prepped meals but more miles driven, adds about 40km 
# Not implemented in code, may be useful if client wants each prep used. 
# Don't need new for loop, just add under previous for j in prep...

#    solver.Add(solver.Sum([segments[(i,j)] for i in carry_in_site_number]) >= 1)

In [11]:
## Results
solver.Solve()
print('Total distance (km) =', solver.Objective().Value())

#Stores assignments in a dictionary
assignments = {}
for i in carry_in_site_number:
    for j in prep_site_number:
        if segments[i, j].solution_value() > 0:
            assignments[(i,j)] = 1
            # print('Carry in site %d assigned to prep site %d.' % (i,j))

Total distance (km) = 132.68026911860102


In [ ]:
# Interesting to look at the spread of meals 
# amount_used()

In [14]:
# Assignments are printed in more easily readable format
prep_list = make_prep_list(assignments)
prep_list

{451: [110, 516, 3831, 4790, 4791, 6040, 6041],
 450: [158, 2121],
 190: [161],
 419: [168, 328],
 437: [178, 252, 473, 509],
 415: [179, 192],
 424: [203, 4880, 4881, 4990, 4991, 5320, 5321],
 971: [223, 533],
 214: [264],
 455: [426, 1170],
 981: [436, 477],
 457: [438],
 150: [497, 891, 5150, 5151],
 412: [605, 5220, 5221],
 292: [750, 3160],
 301: [801],
 405: [999],
 982: [1171],
 408: [1810, 1811],
 218: [2130, 2131, 4780, 4781],
 258: [3161, 3830],
 423: [4900, 4901],
 278: [6020, 6021]}